In [2]:
# Clean slate (optional but helps avoid version pin conflicts)
!pip -q install --upgrade pip

# Correct packages: faiss-CPU and scikit-learn (NOT 'sklearn')
!pip -q install pandas numpy scipy scikit-learn rapidfuzz jellyfish sentence-transformers faiss-cpu networkx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.8 MB/s eta 0:00:00


In [32]:
import pandas as pd, numpy as np, re, math, itertools, networkx as nx
from rapidfuzz import fuzz
import jellyfish
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.mixture import GaussianMixture

# === CONFIG ===
File   = "/content/drive/MyDrive/Colab Notebooks/Projects_Merge.xlsx"   # Oracle side
LEFT_CODE_COL  = "Code"
LEFT_NAME_COL  = "Project Name "
RIGHT_CODE_COL = "Code"
RIGHT_NAME_COL = "Project"

# === LOAD ===
L = pd.read_excel(File, sheet_name='Oracle').fillna("")
R = pd.read_excel(File, sheet_name='MSProject').fillna("")

# keep only needed cols
L = L[[LEFT_CODE_COL, LEFT_NAME_COL]].copy()
R = R[[RIGHT_CODE_COL, RIGHT_NAME_COL]].copy()

L.rename(columns={LEFT_CODE_COL:"code", LEFT_NAME_COL:"name"}, inplace=True)
R.rename(columns={RIGHT_CODE_COL:"code", RIGHT_NAME_COL:"name"}, inplace=True)

L["row_id"] = np.arange(len(L))
R["row_id"] = np.arange(len(R))


In [35]:
# --- Step 2: Compact normalization (replace your previous Step 2) ---
import re, unicodedata

def normalize_compact(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    # lowercase
    s = s.lower().strip()
    # remove accents/diacritics (NFKD)
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    # keep only a–z and 0–9 (drop spaces, / * - ( ) , etc.)
    s = re.sub(r"[^a-z0-9]", "", s)
    return s

def numbers_compact(s: str):
    # pull number groups BEFORE stripping if you prefer; here we extract after compact form
    return re.findall(r"\d+", s or "")

# apply to both dataframes
for df in (L, R):
    # compact canonical form
    df["name_clean"] = df["name"].map(normalize_compact)
    # character-level "tokens" so the DP alignment works at char granularity
    df["name_tokens"] = df["name_clean"].map(list)



In [36]:
# small, fast model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def embed(texts):
    # returns L2-normalized vectors
    X = model.encode(texts, batch_size=256, convert_to_numpy=True, normalize_embeddings=True)
    return X.astype("float32")

X_L = embed(L["name_clean"].tolist())
X_R = embed(R["name_clean"].tolist())

# build FAISS index on R
d = X_R.shape[1]
index = faiss.IndexFlatIP(d)       # cosine since vectors normalized
index.add(X_R)
K = 50  # number of candidates per left record (tune 30–100)

sims, idxs = index.search(X_L, K)  # sims in [-1..1]


In [18]:
def token_sim(a: str, b: str) -> float:
    # character-level similarity per token (0..1)
    return fuzz.QRatio(a, b)/100.0

def affine_align_score(A, B, gap_open=-0.7, gap_ext=-0.15, sub_floor=0.0):
    # A, B are token lists; returns [0..1]
    # substitution score is token_sim; gaps use affine penalties
    n, m = len(A), len(B)
    if n == 0 and m == 0: return 1.0
    if n == 0 or m == 0:  return 0.0

    MINF = -1e9
    M = np.full((n+1, m+1), MINF, dtype=float)  # match/mismatch
    X = np.full((n+1, m+1), MINF, dtype=float)  # gap in A
    Y = np.full((n+1, m+1), MINF, dtype=float)  # gap in B

    M[0,0] = 0.0
    for i in range(1, n+1):
        X[i,0] = gap_open + (i-1)*gap_ext
    for j in range(1, m+1):
        Y[0,j] = gap_open + (j-1)*gap_ext

    for i in range(1, n+1):
        for j in range(1, m+1):
            s = max(token_sim(A[i-1], B[j-1]), sub_floor)  # floor avoids punishing near-matches too hard
            M[i,j] = s + max(M[i-1,j-1], X[i-1,j-1], Y[i-1,j-1])
            X[i,j] = max(M[i-1,j] + gap_open, X[i-1,j] + gap_ext)
            Y[i,j] = max(M[i,j-1] + gap_open, Y[i,j-1] + gap_ext)

    best = max(M[n,m], X[n,m], Y[n,m])

    # normalize by an upper bound (length of longer sequence)
    max_len = max(n, m)
    # assume perfect match token_sim≈1 gives ~1 per step; cap to [0,1]
    return max(0.0, min(1.0, best / max_len))


In [19]:
def numeric_overlap(nums_a, nums_b):
    return 1.0 if set(nums_a) & set(nums_b) else 0.0

def phonetic_agree(p_a, p_b):
    return 1.0 if p_a == p_b and p_a != "" else 0.0

# weights (will auto-calibrate threshold later; you can tweak if needed)
W_EMB   = 0.47
W_ALIGN = 0.47
W_NUM   = 0.06
W_PHON  = 0

pairs = []  # (L_row, R_row, score, components...)
for i in range(len(L)):
    cand_js = idxs[i]
    cand_sims = sims[i]
    A = L.at[i, "name_tokens"]
    for j_idx, emb_sim in zip(cand_js, cand_sims):
        j = int(j_idx)
        B = R.at[j, "name_tokens"]
        align = affine_align_score(A, B)
        # combined
        score = (W_EMB*max(0.0, emb_sim) + W_ALIGN*align + W_NUM*numsc + W_PHON*phons)
        pairs.append((i, j, float(score), float(emb_sim), float(align), float(numsc), float(phons)))

pairs_df = pd.DataFrame(pairs, columns=["L_id","R_id","score","emb","align","num","phon"])


In [21]:
gmm = GaussianMixture(n_components=2, random_state=0).fit(pairs_df[["score"]])
means = gmm.means_.ravel()
order = np.argsort(means)
hi = means[order[1]]
lo = means[order[0]]
# binary search a cut where posteriors equal
def post_p(x):
    p = np.exp(gmm.score_samples(np.array(x).reshape(-1,1)))
    return p
# simple numeric intersection (coarse)
grid = np.linspace(pairs_df["score"].min(), pairs_df["score"].max(), 200)
probs = gmm.predict_proba(grid.reshape(-1,1))
thr = float(grid[np.argmin(np.abs(probs[:,order[1]] - probs[:,order[0]]))])
thr = max(0.70, min(0.98, thr))  # clamp to sane range
thr

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianMixture was fitted with feature names
  warnings.warn(


0.7

In [22]:
# keep only edges above threshold
edges = pairs_df[pairs_df["score"] >= thr]

# Build graph for max weight matching
G = nx.Graph()
for _, r in edges.iterrows():
    G.add_edge(("L", int(r["L_id"])), ("R", int(r["R_id"])), weight=float(r["score"]))

matching = nx.algorithms.matching.max_weight_matching(G, maxcardinality=True)

# Convert to DataFrame
match_rows = []
for a,b in matching:
    if a[0]=="L":
        i, j = a[1], b[1]
    else:
        i, j = b[1], a[1]
    row = edges[(edges["L_id"]==i) & (edges["R_id"]==j)].sort_values("score", ascending=False).iloc[0]
    match_rows.append(row)

match_df = pd.DataFrame(match_rows).sort_values("score", ascending=False).reset_index(drop=True)

# Unmatched left/right
matched_L = set(match_df["L_id"])
matched_R = set(match_df["R_id"])
unmatched_L = [i for i in range(len(L)) if i not in matched_L]
unmatched_R = [j for j in range(len(R)) if j not in matched_R]

len(match_df), len(unmatched_L), len(unmatched_R), float(thr)


(587, 1182, 1242, 0.7)

In [24]:
# Crosswalk with surrogate IDs
match_df["project_id"] = np.arange(1, len(match_df)+1)

# Assign new project_ids for unmatched as well
next_id = len(match_df) + 1
unmatchedL_df = pd.DataFrame({"L_id": unmatched_L, "R_id": [np.nan]*len(unmatched_L),
                              "score": [np.nan]*len(unmatched_L), "project_id": range(next_id, next_id+len(unmatched_L))})
next_id += len(unmatched_L)
unmatchedR_df = pd.DataFrame({"L_id": [np.nan]*len(unmatched_R), "R_id": unmatched_R,
                              "score": [np.nan]*len(unmatched_R), "project_id": range(next_id, next_id+len(unmatched_R))})

crosswalk = pd.concat([match_df[["L_id","R_id","score","project_id"]], unmatchedL_df, unmatchedR_df], ignore_index=True)

# Join codes/names back
cw = crosswalk.merge(L[["row_id","code","name"]], left_on="L_id", right_on="row_id", how="left", suffixes=("","_L"))
cw = cw.merge(R[["row_id","code","name"]], left_on="R_id", right_on="row_id", how="left", suffixes=("","_R"))

cw.rename(columns={"code":"oracle_code","name":"oracle_name","code_R":"excel_code","name_R":"excel_name"}, inplace=True)
cw["match_method"] = np.where(cw["score"].notna(), "auto_alignment", "unmatched")
cw["run_id"] = pd.Timestamp.utcnow().isoformat()
for col in ["oracle_name", "excel_name"]:
    if col in cw.columns:
        cw[col] = cw[col].fillna("").astype(str)

def canonical_name(row):
    a = row.get("oracle_name", "")
    b = row.get("excel_name", "")
    # prefer non-empty; if both non-empty choose the longer
    if a and not b:
        return a
    if b and not a:
        return b
    return a if len(a) >= len(b) else b

cw["canonical_name"] = cw.apply(canonical_name, axis=1)

# Save results
CROSSWALK_CSV = "/content/crosswalk.csv"
MASTER_CSV    = "/content/master_projects.csv"

cw[["project_id","oracle_code","excel_code","oracle_name","excel_name","canonical_name","score","match_method","run_id"]].to_csv(CROSSWALK_CSV, index=False)

master = cw.groupby("project_id").agg({
    "canonical_name":"first",
    "oracle_code":lambda s: ";".join(sorted(set([x for x in s if isinstance(x,str)]))),
    "excel_code": lambda s: ";".join(sorted(set([x for x in s if isinstance(x,str)])))
}).reset_index()

master.to_csv(MASTER_CSV, index=False)

CROSSWALK_CSV, MASTER_CSV


('/content/crosswalk.csv', '/content/master_projects.csv')